In [226]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

from tensorflow.keras import backend as K

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

TensorFlow version is  2.1.0


In [227]:
base_dir = '/Users/lucywang/Documents/Stanford/CS230Project/data_preproc'
train_dir = os.path.join(base_dir, 'train_images_pre')
dev_dir = os.path.join(base_dir, 'dev_images_pre')
test_dir = os.path.join(base_dir, 'test_images_pre')

train_0_dir = os.path.join(train_dir, '0')
print ('Total training class 0 images:', len(os.listdir(train_0_dir)))

train_1_dir = os.path.join(train_dir, '1')
print ('Total training class 1 images:', len(os.listdir(train_1_dir)))

train_2_dir = os.path.join(train_dir, '2')
print ('Total training class 2 images:', len(os.listdir(train_2_dir)))

train_3_dir = os.path.join(train_dir, '3')
print ('Total training class 3 images:', len(os.listdir(train_3_dir)))

train_4_dir = os.path.join(train_dir, '4')
print ('Total training class 4 images:', len(os.listdir(train_4_dir)))

Total training class 0 images: 1264
Total training class 1 images: 259
Total training class 2 images: 699
Total training class 3 images: 135
Total training class 4 images: 207


In [228]:
dev_0_dir = os.path.join(dev_dir, '0')
print ('Total dev class 0 images:', len(os.listdir(dev_0_dir)))

dev_1_dir = os.path.join(dev_dir, '1')
print ('Total dev class 1 images:', len(os.listdir(dev_1_dir)))

dev_2_dir = os.path.join(dev_dir, '2')
print ('Total dev class 2 images:', len(os.listdir(dev_2_dir)))

dev_3_dir = os.path.join(dev_dir, '3')
print ('Total dev class 3 images:', len(os.listdir(dev_3_dir)))

dev_4_dir = os.path.join(dev_dir, '4')
print ('Total dev class 4 images:', len(os.listdir(dev_4_dir)))

Total dev class 0 images: 271
Total dev class 1 images: 56
Total dev class 2 images: 150
Total dev class 3 images: 29
Total dev class 4 images: 44


In [229]:
test_0_dir = os.path.join(test_dir, '0')
print ('Total test class 0 images:', len(os.listdir(test_0_dir)))

test_1_dir = os.path.join(test_dir, '1')
print ('Total test class 1 images:', len(os.listdir(test_1_dir)))

test_2_dir = os.path.join(test_dir, '2')
print ('Total test class 2 images:', len(os.listdir(test_2_dir)))

test_3_dir = os.path.join(test_dir, '3')
print ('Total test class 3 images:', len(os.listdir(test_3_dir)))

test_4_dir = os.path.join(test_dir, '4')
print ('Total test class 4 images:', len(os.listdir(test_4_dir)))

Total test class 0 images: 270
Total test class 1 images: 55
Total test class 2 images: 150
Total test class 3 images: 29
Total test class 4 images: 44


In [230]:
image_size = 224
batch_size = 32

# Rescale all images by 1./255 
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

dev_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size, shuffle = False,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

# Flow dev images in batches using dev_datagen generator
dev_generator = dev_datagen.flow_from_directory(
                dev_dir, # Source directory for the dev images
                target_size=(image_size, image_size),
                batch_size=batch_size, shuffle = False,
                class_mode='categorical')

# Flow test images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the test images
                target_size=(image_size, image_size),
                batch_size=batch_size, shuffle = False,
                class_mode='categorical')

Found 2564 images belonging to 5 classes.
Found 550 images belonging to 5 classes.
Found 548 images belonging to 5 classes.


In [231]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [232]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(5, activation='softmax')
])
model.load_weights("/Users/lucywang/Documents/Stanford/CS230Project/training224_0001Reg01FT2/cp.ckpt")


In [233]:
# Reset generators so images are presented in original order
dev_generator.reset()
test_generator.reset()
train_generator.reset()

In [234]:
# Get model predictions
predictions = model.predict_generator(train_generator)

In [235]:
pred_indices = np.squeeze(np.argmax(predictions,axis=1))
print(pred_indices)

(2564,)


In [237]:
# True labels
truth = np.zeros((270,)) 
truth = np.concatenate((truth,np.ones((55,))))
truth = np.concatenate((truth,np.ones((150,))*2))
truth = np.concatenate((truth,np.ones((29,))*3))
truth = np.concatenate((truth,np.ones((44,))*4))
truth = truth.astype(int)
print(truth)                       

[0 0 0 ... 4 4 4]


In [239]:
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


In [240]:
def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings

In [241]:
def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the quadratic weighted kappa
    quadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator

In [242]:
kappa = quadratic_weighted_kappa(pred_indices, truth, min_rating=None, max_rating=None)
print(kappa)

0.8467750080891039
